# Gramática Libre de Contexto
## Lectura de archivos

In [192]:
file = open(r"C:\Users\hrosa\OneDrive\Documentos\CIC IPN\Primer Semestre\Teoria Computacion\GLC\GLC.txt")

In [193]:
content = file.read()

In [194]:
content.split('\n')

['S -> ABa', 'A -> aab', 'B -> Ac', '', '']

In [195]:
split_content = content.split('\n')
stack = []
for r in split_content:
    rules = []
    for s in r.split():
        rules.append([s])
    stack.append(rules)
print(f'stack: {stack}')

stack: [[['S'], ['->'], ['ABa']], [['A'], ['->'], ['aab']], [['B'], ['->'], ['Ac']], [], []]


In [196]:
#se crean los diccionarios de cada regla de inferencia
rules = {}
for i in stack:
    if not i:
        continue
    head = i[0][0]
    productions = []
    for j in i[2:]:
        if j[0] != '|':
            productions.append(j[0])
            print(productions)
    rules[head] = productions
print(f'rules{rules}')

['ABa']
['aab']
['Ac']
rules{'S': ['ABa'], 'A': ['aab'], 'B': ['Ac']}


## Simplificación de unificación no recursiva

In [197]:
def unificacion_no_recursiva(rules):
    #simplificacion de unificacion no recursiva
    for k,v in rules.items():
        # print(f'k:{k}')
        # print(f'v:{v}')
        first = {}
        for i in v: #por cada elemento de producciones
            max_prefix = i #como no se tiene compun, se toma todo el elemento
            # print(f'HOLA I{i}')
            for j in v: #por cada aparición de elemento: comparar contra los demás
                if j != i: #si esa comparación no es igual
                    # print(f'j:{j} != i:{i}')
                    prefix_common = "" #se inicializa en cero, porque aún no hay prefijo comúb
                    compare = 1 #caracter en común en comparación 
                    #mientras no tenga longitud mayor y haya comun
                    #compare 1: a == a
                    #compare 2: ab != B
                    while compare <= len(i) and compare <= len(j) and i[:compare] == j[:compare]:
                        prefix_common = i[:compare]
                        compare += 1

                    counter = 0
                    for production in v: #cuenta # de elementos con prefijo compun
                        if production.startswith(prefix_common):
                            counter += 1
                    if prefix_common and counter > 1: #actualizar el término más común
                        max_prefix = prefix_common

            prefix_found = max_prefix
            body = i[len(prefix_found):] #no incluir el prefijo común

            if prefix_found not in first:
                first[prefix_found] = []
            first[prefix_found].append(body)
    #     print(first)

    # print(f'new rules: {first}')
    new_rules = {}
    counter_r = 1
    # print(rules)
    origin_rules = None
    for k, v in rules.items():
        origin_rules = k

    productions_origin = []

    for k,v in first.items():
        if len(v) == 1: #si solo hay una producción:
            new_rules[k] = v
            productions_origin.append(k+v[0])
        else:
            aux = f"R{counter_r}"
            counter_r += 1
            new_rules[aux] = v
            productions_origin.append(k+aux)

    rules_unified = {origin_rules:productions_origin}
    for k,v in new_rules.items():
        rules_unified[k] = v

    # print(f'new rules: {rules_unified}'
    return rules_unified

## Simplicidad de símbolos redundantes
1. variables no productivas.
2. variables inalcanzables.

In [198]:
def simbolos_redundantes(rules):
    not_productive = []
    for k,v in rules.items():
        # print(f'k:{k} and v:{v}')
        productive = False

        for production in v:
            terminal_count = 0
            no_terminal_count = False

            for char in production:
                if char.islower(): #es terminal?
                    terminal_count += 1
                if char.isupper(): #es no terminal
                    no_terminal_count = True

            if terminal_count >= 1 and not no_terminal_count:
                productive = True
                
        if not productive and k != 'S':
            not_productive.append(k)
    # print(not_productive)

    #=======
    filter_no_productive = {}
    for k,v in rules.items():
        if k not in not_productive:
            new_rules = []
            for production in v:
                productive = True
                for char in production:
                    if char in not_productive:
                        # print(char)
                        productive = False
                if productive:
                    new_rules.append(production)
            if new_rules:
                filter_no_productive[k] = new_rules
    # print(filter_no_productive)

    #====
    if 'S' not in filter_no_productive:
        return rules
    else:
        rule_s = filter_no_productive['S']
    # print(f'rules s: {rule_s}')
    not_redundants = {}
    inalcanzable = []

    for k, v in filter_no_productive.items():
        alcanzable = False
        for production in v:
            # print(production)
            # print(f'k:{k}')
            for i in rule_s:          
                # print(i)
                # print(f'---{k}')
                if k in i or k == 'S':
                    alcanzable = True
            if not alcanzable:
                inalcanzable.append(k)
    # print(inalcanzable)

    for k,v in filter_no_productive.items():
        if k not in inalcanzable:
            not_redundants[k] = v
    # print(not_redundants)

    return not_redundants

## Simplicidad por reglas unitarias

In [199]:
def simplicidad_unitaria(rules):
    unitaria_terminal = {}
    for key in rules:
        unitaria = list(key)
        contador = 0
        while contador < len(unitaria):
            B = unitaria[contador]
            if B in rules:
                for production in rules[B]:
                    if len(production) == 1 and production.isupper() and production not in unitaria:
                        unitaria.append(production)
            contador += 1
        unitaria_terminal[key] = unitaria
    
    # print(f'unitaria terminal: {unitaria_terminal}')
    new_rules = {}
    for key in rules:
        new_rules[key] = []
        for i in unitaria_terminal[key]:
            if i in rules:
                for production in rules[i]:
                    if not(len(production) == 1 and production.isupper()):
                        new_rules[key].append(production)

    # print(f'new rules: {new_rules}')
    no_unitarias = ['S']
    stack = ['S']

    while stack:
        current_value = stack.pop()
        if current_value in new_rules:
            for production in new_rules[current_value]:
                for char in production:
                    if char.isupper() and char not in no_unitarias:
                        no_unitarias.append(char)
                        stack.append(char)
    # print(f'stack:{stack}')
    # print(f'no unitary: {no_unitarias}')
    final = {}

    for key in no_unitarias:
        if key in new_rules:
            final[key] = new_rules[key]

    return final

## Simplicidad por epsilon-producciones

In [200]:
def epsilon_producciones(rules):
    epsilon = []
    #Hay epsilon en los valores?
    for key, value in rules.items():
        if 'epsilon' in value and key not in epsilon:
            epsilon.append(key)

    change = True
    while change:
        change = False

        for key, value in rules.items():
            if key in epsilon:
                continue
            
            epsilons = True
            for production in value:
                for char in production:
                    if char not in epsilon:
                        epsilons = False
                        break
                if epsilons:
                    if key not in epsilon:
                        epsilon.append(key)
                        change = True
    
    new_rules = {}
    for key, value in rules.items():
        new_rules[key] = []
        for production in value:
            if production != 'epsilon':
                # print('yes')
                if production not in new_rules[key]:
                    new_rules[key].append(production)
            
            new_productions = [""]

            for char in production:
                p = []
                if char in epsilon:
                    for i in new_productions:
                        p.append(i)
                        p.append(i+char)
                else:
                    for i in new_productions:
                        p.append(i+char)
                
                new_productions = p
            
            for i in new_productions:
                if i and i != 'epsilon' and i not in new_rules[key]:
                    new_rules[key].append(i) 
    return new_rules

## Forma Normal de Chomsky (FNC)

1. Identificar y eliminar recursión izquierda (inmediata e indirecta)

In [201]:
def recursividad_indirecta(rules):
    no_terminales = list(rules.keys())

    for i in range(len(no_terminales)):
        ai = no_terminales[i]
        new_productions = []

        for production in rules[ai]: #cada produccion de regla[0]
            first = production
            
            if first in no_terminales[:i]:
                aj = first
                for ij in rules[aj]:
                    new_productions.append(ij + production[len(aj):])
            else:
                new_productions.append(production)

        rules[ai] = new_productions

    return recursividad(rules)

def recursividad(rules):
    new_rules = {}
    abcd = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]
    letra_ocupada = []
    for key, value in rules.items():
        letra_ocupada.append(key)
        recursiva = []
        no_recursiva = []

        for production in value:
            if production.startswith(key):
                recursiva.append(production[len(key):])
            else:
                no_recursiva.append(production)

        if recursiva:
            new_upper = None
            for i in abcd:
                if i not in letra_ocupada:
                    new_upper = i
                    letra_ocupada.append(i)
                    break

            new_rules[key] = []
            for i in no_recursiva:
                new_rules[key].append(i + new_upper)

            new_rules[new_upper] = []
            for j in recursiva:
                new_rules[new_upper].append(j + new_upper)
            new_rules[new_upper].append('epsilon')

        else:
            new_rules[key] = value

    return new_rules

In [202]:
sin_recursividad = recursividad_indirecta(rules)
# new_rules

In [203]:
sin_recursividad

{'S': ['ABa'], 'A': ['aab'], 'B': ['Ac']}

2. Simplificación de reglas

In [204]:
for key, value in sin_recursividad.items():
    new_productions = []
    for production in value:
        if production.startswith('epsilon') and len(production) > len('epsilon'):
            new_productions.append(production[len('epsilon'):])
        else:
            new_productions.append(production)
    sin_recursividad[key] = new_productions

In [205]:
sin_recursividad

{'S': ['ABa'], 'A': ['aab'], 'B': ['Ac']}

In [206]:
# simbolos_redundantes(simplicidad_unitaria(epsilon_producciones(sin_recursividad)))

In [207]:
changes = True
simplificacion_prev = None
while changes:
    simplificacion = simbolos_redundantes(simplicidad_unitaria(epsilon_producciones(sin_recursividad)))
    if simplificacion == simplificacion_prev:
        changes = False
    else:
        changes = True
        simplificacion_prev = simplificacion

simplificacion


{'S': ['ABa'], 'A': ['aab'], 'B': ['Ac']}

3. Substituir terminales No Independientes

In [208]:
terminales_independientes = {}
for key, value in simplificacion.items():
    for production in value:
        for char in production:
            if not char.isupper():
                terminales_independientes[f'Z{char.lower()}'] = char

print(terminales_independientes)

{'Za': 'a', 'Zb': 'b', 'Zc': 'c'}


In [209]:

new_rules = {}
for key, value in simplificacion.items():
    new_productions = []
    for production in value:
        new_production = ""
        for char in production:
            for kt, vt in terminales_independientes.items():
                # print(f'production.{production} and len:{len(production)}')
                if char == vt and len(production)!=1:
                    char = kt
            new_production += char + '#'
        new_productions.append(new_production)
    new_rules[key] = new_productions

In [210]:
new_rules

{'S': ['A#B#Za#'], 'A': ['Za#Za#Zb#'], 'B': ['A#Zc#']}

In [211]:
new_rules.update(terminales_independientes)

In [212]:
for key, value in new_rules.items():
    if not isinstance(value,list):
        new_rules[key] = [value]
new_rules

{'S': ['A#B#Za#'],
 'A': ['Za#Za#Zb#'],
 'B': ['A#Zc#'],
 'Za': ['a'],
 'Zb': ['b'],
 'Zc': ['c']}

4. Substituir producciones con longitud mayor o igual a 3

In [213]:
# import re
# for key, value in new_rules.items():
#     for production in value:
#         token = re.findall(r'Z[a-z]',production)
#         if token:
#             z_element = token
#             # print(z_element)

In [214]:
new_list = {}
for key, value in new_rules.items():
    # print(f'key:{key} and value:{value}')
    new_productions = []
    for production in value:
        char = production.split('#')
        cleaned = []
        for i in char:
            # print(i)
            if i != '':
                cleaned.append(i)
        new_productions.append(cleaned)
    new_list[key] = new_productions
new_list


{'S': [['A', 'B', 'Za']],
 'A': [['Za', 'Za', 'Zb']],
 'B': [['A', 'Zc']],
 'Za': [['a']],
 'Zb': [['b']],
 'Zc': [['c']]}

In [215]:
for key, value in new_list.items():
    print(f'key:{key} and value:{value}')

key:S and value:[['A', 'B', 'Za']]
key:A and value:[['Za', 'Za', 'Zb']]
key:B and value:[['A', 'Zc']]
key:Za and value:[['a']]
key:Zb and value:[['b']]
key:Zc and value:[['c']]


In [216]:
final_fnc = {}
counter_r = 1

for key, value in new_list.items():
    for production in value:
        # print(f'key:{key} and prod:{production}')
        if len(production) >= 3:
            print(f'yes I am {production} and my father is {key}')
            aux = f'R{counter_r}'
            counter_r += 1
            final_fnc[key] = [production[0] + aux]
            new_body = production[1:]
            # print(new_body)

            while len(new_body) >= 3:
                # new_aux = f'R{counter_r}'
                counter_r += 1
                final_fnc[key] = [production[0] + aux]
                # aux = new_aux
                new_body = new_body[1:]
            final_fnc[aux] = new_body
        else:
            if key not in final_fnc:
                final_fnc[key] = []
            final_fnc[key].append(production)
# final_fnc
    # print(f'production:{value} and len:{len(value[0])}')
    # if len(value[0]) >= 3:
    #     print()
    #     print(value[0])
    #     aux = f'R{counter_r}'
    #     counter_r += 1
    #     final_fnc[key] = value[0][0] + aux
    #     body = value[0][1:]

    #     while len(body) >= 3:
    #         new_aux = f'R{counter_r}'
    #         final_fnc[aux] = body[0][0] + new_aux
    #         aux = new_aux
    #         body = body[0][1:]
    #         counter_r += 1

    #     final_fnc[aux] = body
    # else:
    #     final_fnc[key] = value

yes I am ['A', 'B', 'Za'] and my father is S
yes I am ['Za', 'Za', 'Zb'] and my father is A


In [224]:
final_fnc

{'S': ['AR1'],
 'R1': ['B', 'Za'],
 'A': ['ZaR2'],
 'R2': ['Za', 'Zb'],
 'B': [['A', 'Zc']],
 'Za': [['a']],
 'Zb': [['b']],
 'Zc': [['c']]}

In [237]:
for key, value in final_fnc.items():
    str_value = []
    for production in value:
        if isinstance(production, list):
            str_value.append("".join(map(str,production)))
        else:
            str_value.append(str(production))
    print(f'{key}->{' | '.join(str_value)}')

S->AR1
R1->B | Za
A->ZaR2
R2->Za | Zb
B->AZc
Za->a
Zb->b
Zc->c


## Forma Normal de Greibach (FNG)

1. Tranformar gramática en FNC.

In [ ]:
final_fnc = {'S': ['AA', 'a'], 'A': ['SS', 'b']}
# final_fnc = {'S': ['AB', 'BC'], 'A': ['aB', 'bA', 'a'], 'B': ['bB', 'cC', 'b'], 'C': ['c']}
# final_fnc = {'S': ['ABa'], 'A': ['aab'], 'B': ['Ac']}


2. Renombrar las variables en estricto orden.


In [ ]:
renombrar = {}
counter_terminal = 2
for key, value in final_fnc.items():
    for production in value:
        for char in production:
            # print(char)
            if char == 'S':
                renombrar['A1'] = char
            # elif key == 'S': #standby
            #     renombrar['A1'] = 'S'
            elif char.isupper():
                if char not in renombrar.values():
                    renombrar[f'A{counter_terminal}'] = char
                    counter_terminal += 1
                continue
            # counter_terminal += 1
print(renombrar)

In [ ]:
new_rules = {}
for key, value in final_fnc.items():
    new_productions = []
    for production in value:
        new_production = ""
        for char in production:
            for kt, vt in renombrar.items():
                if char == vt:
                    char = kt
                if key == vt:
                    key = kt
            new_production += char
        new_productions.append(new_production)
    new_rules[key] = new_productions

In [ ]:
new_rules

3. Identificar las reglas Ai -> aAj donde j"<"i <br>
A2 -> A1A1

In [ ]:
import re
rules_identified = {}
for key, value in new_rules.items():
    i = int(key[1:])
    for production in value:
        number_a = re.findall(r'^A(\d+)',production)
        if not number_a:
            continue

        j = int(number_a[0])
        if j < i:
            if key in rules_identified:
                rules_identified[key].append(production)
            else:
                rules_identified[key] = [production]

In [ ]:
rules_identified

4. Transformar al sustituir las reglas.
